In [1]:
!pip install beautifulsoup4 
!pip install langchain langchain-community 
!pip install chromadb 
!pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [52]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_together import ChatTogether, TogetherEmbeddings
from llama_cpp import Llama  # Import TogetherEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("API_KEY")


#### INDEXING ####


In [35]:

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print(len(docs))


1


### Split

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)


## Markdown

In [41]:
embedding_model = TogetherEmbeddings(
    together_api_key = API_KEY,
    model="togethercomputer/m2-bert-80M-8k-retrieval",  
    request_timeout=30.0,  
    max_retries=3,
    
)

In [42]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [44]:
retriever = vectorstore.as_retriever()


In [45]:
prompt = hub.pull("rlm/rag-prompt")


In [47]:
# LLM
llm = ChatTogether(
        together_api_key= API_KEY,
        model="meta-llama/Llama-3-70b-chat-hf",
    )

In [48]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [49]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
ans = rag_chain.invoke("What is ScaNN?")
print(ans)

ScaNN is a method for efficient vector similarity search, as announced in a Google Blog post on July 28, 2020. It is a technique for fast vector search. I don't have more information about ScaNN beyond this.
